### Regularized (banded) CV regression workflow for Neuroscout
This notebook implements an encoding model for a single subject using Regularized Ridge Regression, as implemented in https://github.com/gallantlab/himalaya. In Neuroscout, this same pipeline should be run for all subjects.
- Input needed from the user
    - Define datasets (independent model fitting for all datasets)
    - Define cross-validation strategy
        - Across runs
        - Within runs
    - Define estimator
    - Define preprocessing steps (e.g., scaling?)
    - Define bands
    - Pass parameters
    - Output: scores, parameters, predicted time series
- Define outputs

<b> To do<b>:
- Decide which outputs to store
- Implement

In [21]:
import pyns
import pandas as pd
import nibabel as nib
import numpy as np
import glob
from copy import deepcopy
from pathlib import Path

In [2]:
api = pyns.Neuroscout()

In [3]:
dataset_name = 'Budapest'

### Choose subject

Here, we can explore the runs available in this dataset. Let's choose the first subject we see, and analyze all of their runs

In [22]:
# Select subject from first run available in dataset
api.runs.get(dataset_name='Budapest')[0]

{'acquisition': None,
 'dataset_id': 27,
 'duration': 535.0,
 'id': 1435,
 'number': 3,
 'session': None,
 'subject': 'sid000005',
 'task': 48,
 'task_name': 'movie'}

In [24]:
subject = api.runs.get(dataset_name='Budapest')[0]['subject']
subject

'sid000005'

### Fetch predictors from Neuroscout and create design matrix
Let's retrieve predictor events for multiple sets of predictors. \
For now, let's pick two sets: <b>MFCC</b> + <b>mel</b> features (plus some confounds).

In [25]:
mfccs = [f'mfcc_{i}' for i in range(20)]
mel = [f'mel_{i}' for i in range(64)]
confounds = ['rot_x', 'rot_y', 'rot_z', 'trans_x', 'trans_y', 'trans_z',
             'a_comp_cor_00', 'a_comp_cor_01', 'a_comp_cor_02',
             'a_comp_cor_03','a_comp_cor_04','a_comp_cor_05']

all_vars = mfccs + mel + confounds

In [26]:
from pyns.fetch_utils import fetch_neuroscout_predictors

`fetch_neuroscout_predictors` will retrive the named predictors from the Neurscout API, and (optionally) resampled them to `TR`. All timepoints are concatenated into a single file, with identifying columns (i.e. `subject`, `run`)

In [8]:
X_vars = fetch_neuroscout_predictors(
    predictor_names=all_vars, dataset_name=dataset_name, subject=subject, 
    resample=True, return_type='df')

/home/alejandro/repos/neuroscout-encoding-models/src/pyns/pyns/endpoints/base.py:135: UserWarning: No API endpoint for stimulus_id, could not convert
  warnings.warn(f"No API endpoint for {col}, could not convert")
/home/alejandro/repos/neuroscout-encoding-models/src/pyns/pyns/fetch_utils.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['amplitude'] = pd.to_numeric(df['amplitude'])


In [31]:
X_vars.head()

,run,onset,subject,duration,a_comp_cor_00,a_comp_cor_01,a_comp_cor_02,a_comp_cor_03,a_comp_cor_04,a_comp_cor_05,...,mfcc_7,mfcc_8,mfcc_9,rot_x,rot_y,rot_z,trans_x,trans_y,trans_z,run_id
0,1,0.0,sid000005,1.0,0.000340,0.000285,-0.000761,-0.000265,-0.000440,0.000032,...,-0.000051,-0.000091,-0.000061,0.000118,0.000052,-2.547634e-07,0.000171,0.005245,0.000334,1433
1,1,1.0,sid000005,1.0,-0.000940,-0.000778,0.002106,0.000765,0.001213,-0.000080,...,0.000075,0.000210,0.000122,0.000056,0.000028,7.858808e-07,-0.000472,-0.004865,-0.004346,1433
2,1,2.0,sid000005,1.0,0.002645,0.002194,-0.005917,-0.002099,-0.003416,0.000216,...,-0.000039,-0.000465,-0.000217,-0.000074,-0.000012,-2.307046e-06,0.003043,-0.014846,0.000742,1433
3,1,3.0,sid000005,1.0,-0.018634,-0.017725,0.040803,0.005202,0.024639,-0.002898,...,-0.000306,0.000970,0.000288,-0.000394,0.000022,6.461683e-06,0.002958,0.047416,0.011739,1433
4,1,4.0,sid000005,1.0,-0.032204,-0.041976,0.062628,-0.043931,0.045983,-0.010824,...,0.001847,-0.001848,-0.000011,-0.000042,-0.000134,-1.810310e-05,-0.001411,0.007173,0.002242,1433


### Fetch fMRI data and load images

To retrieve Neuroscout data, we use `datalad`. 
We can determine the correct URL for the preprocessed dataset using the Neuroscout API


In [35]:
from datalad.api import install, get
from bids.layout import BIDSLayout

In [39]:
preproc_address = api.datasets.get(name=dataset_name)[0]['preproc_address']
preproc_address

'https://github.com/neuroscout-datasets/ds003017.git'

In [42]:
# Set up local paths
local_datasets = Path('/media/neuroscout-data/neuroscout/datasets/neuroscout-datasets/')
dataset_path = local_datasets / dataset_name

# Install DataLad dataset if dataset does not exist locally
if not dataset_path.exists():
    install(path=local_dataset_path, source=preproc_address)

Here, we'll use pybids to identify the file we need to fetch, and use DataLad to fetch them

In [52]:
# TODO: Potentially build this into pyns using pybids, or a simpler logic
layout = BIDSLayout(dataset_path / 'fmriprep', derivatives=dataset_path / 'fmriprep', index_metadata=False)
# Identify functional runs
subject_images = layout.get(subject=subject, desc='preproc', extension='.nii.gz', suffix='bold')
subject_images

/home/alejandro/anaconda3/lib/python3.7/site-packages/bids/layout/validation.py:51: UserWarning: The ability to pass arguments to BIDSLayout that control indexing is likely to be removed in future; possibly as early as PyBIDS 0.14. This includes the `config_filename`, `ignore`, `force_index`, and `index_metadata` arguments. The recommended usage pattern is to initialize a new BIDSLayoutIndexer with these arguments, and pass it to the BIDSLayout via the `indexer` argument.
  warnings.warn("The ability to pass arguments to BIDSLayout that control "
/home/alejandro/anaconda3/lib/python3.7/site-packages/bids/layout/validation.py:156: UserWarning: The PipelineDescription field was superseded by GeneratedBy in BIDS 1.4.0. You can use ``pybids upgrade`` to update your derivative dataset.
  warnings.warn("The PipelineDescription field was superseded "


[<BIDSImageFile filename='/media/neuroscout-data/neuroscout/datasets/neuroscout-datasets/Budapest/fmriprep/sub-sid000005/func/sub-sid000005_task-movie_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'>,
 <BIDSImageFile filename='/media/neuroscout-data/neuroscout/datasets/neuroscout-datasets/Budapest/fmriprep/sub-sid000005/func/sub-sid000005_task-movie_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'>,
 <BIDSImageFile filename='/media/neuroscout-data/neuroscout/datasets/neuroscout-datasets/Budapest/fmriprep/sub-sid000005/func/sub-sid000005_task-movie_run-3_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'>,
 <BIDSImageFile filename='/media/neuroscout-data/neuroscout/datasets/neuroscout-datasets/Budapest/fmriprep/sub-sid000005/func/sub-sid000005_task-movie_run-4_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'>,
 <BIDSImageFile filename='/media/neuroscout-data/neuroscout/datasets/neuroscout-datasets/Budapest/fmriprep/sub-sid000005/func/sub-sid000005_task-movie_run-5

In [47]:
# Fetching using DataLad (if needed)
# get([f.path for f in subject_images, dataset=dataset_path)

Finally, to prepare the images for analysis, we'll load them into a single array, and accompanying dataframe with meta-data for every volume

In [49]:
def _stack_images(image_objects):
    """ Stack images into single array, and collect metadata entities into dataframe """
    arrays = []
    entities = []
    image_objects = sorted(image_objects, key=lambda x: x.entities['run'])
    for img in image_objects:
        data = np.asanyarray(nib.load(img.path).dataobj)
        run_y = data.reshape([data.shape[0] * data.shape[1] * data.shape[2], data.shape[3]]).T
        arrays.append(run_y)
        entities += [dict(img.entities)] * run_y.shape[0]
    entities = pd.DataFrame(entities)
    return np.vstack(arrays), entities

In [51]:
Y, img_idx = _stack_images(subject_images)

In [56]:
# Image volume
Y[0:100]

array([[-0.00376109, -0.00376109, -0.00376109, ..., -0.00376109,
        -0.00376109, -0.00376109],
       [-0.00376109, -0.00376109, -0.00376109, ..., -0.00376109,
        -0.00376109, -0.00376109],
       [-0.00376109, -0.00376109, -0.00376109, ..., -0.00376109,
        -0.00376109, -0.00376109],
       ...,
       [-0.00376109, -0.00376109, -0.00376109, ..., -0.00376109,
        -0.00376109, -0.00376109],
       [-0.00376109, -0.00376109, -0.00376109, ..., -0.00376109,
        -0.00376109, -0.00376109],
       [-0.00376109, -0.00376109, -0.00376109, ..., -0.00376109,
        -0.00376109, -0.00376109]])

In [54]:
# Meta-data
img_idx.head()

,datatype,desc,extension,run,space,subject,suffix,task
0,func,preproc,.nii.gz,1,MNI152NLin2009cAsym,sid000005,bold,movie
1,func,preproc,.nii.gz,1,MNI152NLin2009cAsym,sid000005,bold,movie
2,func,preproc,.nii.gz,1,MNI152NLin2009cAsym,sid000005,bold,movie
3,func,preproc,.nii.gz,1,MNI152NLin2009cAsym,sid000005,bold,movie
4,func,preproc,.nii.gz,1,MNI152NLin2009cAsym,sid000005,bold,movie


### Preprocessing and model fitting

Run the encoding model with lots of comments and printing.

In [136]:
from sklearn.model_selection import KFold, GroupKFold, PredefinedSplit
from himalaya.ridge import GroupRidgeCV
from himalaya.scoring import correlation_score

Cross-validated model fitting, prediction, and scoring loosely based on scikit-learn's [`cross_val_score`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html). Returns a `results` dictionary with `'coefficients'`, `'test_predictions'`, and `'test_scores'` keys containing lists of numpy arrays for each outer cross-validation fold.

In [308]:
def _model_cv(estimator, X_vars, y, bands=None, groups=None,
              scoring=correlation_score, cv=None,
              inner_cv=None, confounds=None, split=None):
    # Container for results
    results = {
        'coefficients': [],
        'test_predictions': [],
        'test_scores': []}
    
    # If confounds, stack at the end
    if confounds is not None:
        bands.append(confounds)
    
    if bands is not None:
        X = []
        for band in bands:
            X.append(X_vars[band].as_matrix())
    else:
        X = X_vars.as_matrix()

    # Extract number of samples for convenience
    n_samples = y.shape[0]
    
    # Set default cross-validation to KFold if not specified
    cv = KFold() if not cv else cv
    
    # Loop through outer cross-validation folds
    for train, test in cv.split(np.arange(n_samples), groups=groups):
        
        # Get training model for list of model bands
        X_train = [x[train] for x in X] if type(X) == list else X[train]
        X_test = [x[test] for x in X] if type(X) == list else X[test]
        
        # Create inner cross-validation loop if specified
        if inner_cv:
            # Split inner cross-validation with groups if supplied
            inner_groups = np.array(groups)[train] if groups else groups
            inner_splits = inner_cv.split(np.arange(n_samples)[train],
                                          groups=inner_groups)
            
            # Update estimator with inner cross-validator
            estimator.set_params(cv=inner_splits)
            print(np.unique(inner_groups))
        
        # Fit the regression model on training data
        estimator.fit(X_train, y[train])
        
        # Zero out coefficients for confounds if provided
        if confounds is not None:
            estimator.coef_[-len(confounds):] = 0
        
        # Compute predictions with optional splitting by band
        kwargs = {}
        if split is not None:
            kwargs['split'] = split
        test_prediction = estimator.predict(X_test, **kwargs)
        
        # Test scores should also optionally split by band
        test_score = scoring(y[test], test_prediction)
        
        # Populate results dictionary
        results['coefficients'].append(estimator.coef_)
        results['test_predictions'].append(test_prediction)
        results['test_scores'].append(test_score)
        
    return results

In [292]:
# Convert input data to numpy and fill NaNs
y = Y[:, :100]

# Default estimator should be GroupRidgeCV
estimator = GroupRidgeCV(groups='input')

# Default cross-validation should be leave-one-run-out
n_runs = len(idx['run_id'].unique())
cv = GroupKFold(n_splits=n_runs)
inner_cv = GroupKFold(n_splits=n_runs - 1)

In [21]:
# Run model with specified cross-validation, groups, confounds, and split outputs
results = _model_cv(estimator, X_vars, y, cv=cv, inner_cv=inner_cv, groups=X_metadata['run_id'].tolist(), split=True)

[1433 1434 1435 1436]
[........................................] 100% | 2.25 sec | 100 random sampling with cv | 
[1433 1434 1435 1437]
[........................................] 100% | 2.56 sec | 100 random sampling with cv | 
[1434 1435 1436 1437]
[........................................] 100% | 2.71 sec | 100 random sampling with cv | 
[1433 1434 1436 1437]
[........................................] 100% | 2.81 sec | 100 random sampling with cv | 
[1433 1435 1436 1437]
[........................................] 100% | 2.58 sec | 100 random sampling with cv | 


In [303]:
X_metadata['run_id'].tolist()

[1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,
 1433,

In [22]:
results = _model_cv(estimator, X, y, cv=cv, inner_cv=inner_cv, groups=idx, confounds=confounds)

[1433 1434 1435 1436]
[........................................] 100% | 2.01 sec | 100 random sampling with cv | 
[1433 1434 1435 1437]
[........................................] 100% | 2.28 sec | 100 random sampling with cv | 
[1434 1435 1436 1437]
[........................................] 100% | 2.37 sec | 100 random sampling with cv | 
[1433 1434 1436 1437]
[........................................] 100% | 2.49 sec | 100 random sampling with cv | 
[1433 1435 1436 1437]
[........................................] 100% | 2.77 sec | 100 random sampling with cv | 


In [23]:
results = _model_cv(estimator, X, y, cv=KFold(), inner_cv=KFold())

[None]
[........................................] 100% | 2.57 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.67 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.38 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.73 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.68 sec | 100 random sampling with cv | 


In [127]:
results = _model_cv(estimator, X, y, cv=KFold())

[........................................] 100% | 0.10 sec | 100 random sampling with cv | 
[..................................      ] 86% | 0.10 sec | 100 random sampling with cv | 

/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


[........................................] 100% | 0.12 sec | 100 random sampling with cv | 
[........................................] 100% | 0.11 sec | 100 random sampling with cv | 
[..................................      ] 85% | 0.09 sec | 100 random sampling with cv | 

/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


[........................................] 100% | 0.10 sec | 100 random sampling with cv | 
[........................................] 100% | 0.10 sec | 100 random sampling with cv | 


/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


In [128]:
results = _model_cv(estimator, X, y, inner_cv=KFold())

[None]
[........................................] 100% | 2.77 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.67 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.78 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.69 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.65 sec | 100 random sampling with cv | 


In [117]:
results = _model_cv(estimator, X, y, groups=idx)

[........................................] 100% | 0.10 sec | 100 random sampling with cv | 
[...................................     ] 89% | 0.09 sec | 100 random sampling with cv | 

/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


[........................................] 100% | 0.10 sec | 100 random sampling with cv | 
[........................................] 100% | 0.12 sec | 100 random sampling with cv | 
[............................            ] 71% | 0.08 sec | 100 random sampling with cv | 

/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


[........................................] 100% | 0.11 sec | 100 random sampling with cv | 
[........................................] 100% | 0.11 sec | 100 random sampling with cv | 


/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


In [309]:
# Using single-band (non-banded) model with sklearn RidgeCV
from sklearn.linear_model import RidgeCV
results = _model_cv(RidgeCV(), X_vars, y, cv=cv, inner_cv=inner_cv, groups=X_metadata['run_id'].tolist())

/home/alejandro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


[1433 1434 1435 1436]
[1433 1434 1435 1437]
[1434 1435 1436 1437]
[1433 1434 1436 1437]
[1433 1435 1436 1437]


### Handling outputs

In [ ]:
### 

### Validate against other workflows

In [ ]:
### 